## 📦 Задеплой Агента с помощью `LangServe`. 🚀</center>




Представим, что у нас есть настроеный работающий агент, и мы хотим, чтобы другие люди тоже могли им воспользоваться. 
Самое время задеплоить его на сервере и дать ощутить пользу от него другим людям.

Фрэймворк `LangServe` позволяет деплоить Langchain-цепи, агентов и любые `runnables` (протокол кастомных цепей), как `REST API`-сервисы и даже делает для них простую веб-обертку (`playground`).

`LangServe` интегрирован с `FastAPI` ([документация](https://fastapi.tiangolo.com/)) и для валидации данных использует `pydantiс` ([документация](https://docs.pydantic.dev/latest/)).

<div class="alert alert-info">
    
В качестве сервера будем использовать код из ячейки ниже. 
Чтобы запустить сервер локально нужно сохранить код ячейки в питоновский файл `.py` и вызвать через командную строку командой `python файл.py`. 

Сервер будет слушать запросы по адресу `"0.0.0.0:8501"` (в нашем случае).
У нас будет 2 эндпоинта `/joke` и `/rag_agent`. 
* По первому у нас будет доступ к цепочке, которая будет выдавать шутки на определенную тему (тему мы подадим post-запросом с клиента).
* По второму - знакомый нам RAG-agent для ответов на вопросы о пиццерии.

In [ ]:
%%writefile server.py
#!/usr/bin/env python

from fastapi import FastAPI
from utils import ChatOpenAI, OpenAIEmbeddings
from typing import Any

from langserve import add_routes
from langchain.pydantic_v1 import BaseModel
from langchain.document_loaders import WebBaseLoader
from langchain import hub
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent

class Input(BaseModel):
    input: str


class Output(BaseModel):
    output: Any

def cut_output(output):
    return output['output']


loader = WebBaseLoader("https://allopizza.su/spb/kupchino/about")
data = loader.load()

course_api_key = ''# ключ курса (если используем ключи из курса)
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
embeddings = OpenAIEmbeddings(course_api_key=course_api_key)
db_embed = FAISS.from_documents(texts, embeddings)
retriever = db_embed.as_retriever()

tool = create_retriever_tool(
    retriever, # наш ретривер
    "search_web", # имя инструмента
    "Searches and returns data from page", # описание инструмента подается в ЛЛМ
)

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, [tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[tool])

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output) | cut_output,
    path="/rag_agent",  # эндпоинт для rag агента
)

prompt2 = ChatPromptTemplate.from_template("tell me a joke about {topic}")
add_routes(
    app,
    prompt2 | llm,
    path="/joke",  # эндпоинт для цепочки
)



if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8501)

Overwriting server.py



    
В ячейках ниже - код клиента.
Мы посылаем POST-запрос на нужный адрес.  

In [ ]:
# Проверяем "шуточную" цепочку
import requests

response = requests.post(
    "http://0.0.0.0:8501/joke/invoke",
    json={'input': {'topic': 'robot'}}
)
response.json()

{'output': {'content': 'Why did the robot go on a diet? Because it had a byte problem!',
  'additional_kwargs': {},
  'response_metadata': {'token_usage': {'completion_tokens': 16,
    'prompt_tokens': 13,
    'total_tokens': 29},
   'model_name': 'gpt-3.5-turbo',
   'system_fingerprint': 'fp_3b956da36b',
   'finish_reason': 'stop',
   'logprobs': None},
  'type': 'ai',
  'name': None,
  'id': 'run-e2c75e33-432f-4d3f-820b-9c8f38a900b1-0',
  'example': False},
 'metadata': {'run_id': '5d41242a-19da-4e84-a336-b1b49a36dcdc',
  'feedback_tokens': []}}

In [ ]:
# Проверяем RAG-агента
response = requests.post(
    "http://0.0.0.0:8501/rag_agent/invoke",
    json={'input': {'input':'Из скольки авто состоит автопарк компании?'}}
)
response.json()

{'output': {'output': 'Автопарк компании состоит из более чем 200 легковых автомобилей.'},
 'metadata': {'run_id': '79d2a9fb-d8e0-4af6-b006-6953635fd31f',
  'feedback_tokens': []}}